In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
import os

# Example mappings (adjust according to your actual class_indices)
skin_concern_classes = {0: 'acne', 1: 'blackheads', 2: 'redness'}
skin_type_classes = {0: 'oily', 1: 'dry', 2: 'normal', 3: 'sensitive', 4: 'combination'}

from dotenv import load_dotenv
load_dotenv()
FOLDER_PATH = os.getenv("FOLDER_PATH")

if not FOLDER_PATH:
    raise ValueError("FOLDER_PATH not set in .env file!")

skin_concern_model = load_model(os.path.join(FOLDER_PATH, 'models', 'skin_concern_classifier_PT.h5'))
skin_type_model = load_model(os.path.join(FOLDER_PATH, 'models', 'skin_type_classifier_PT.h5'))

def preprocess_image(img_path, target_size):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict_skin_profile_dual_input(
    img_path,
    concern_model, concern_input_size, concern_classes,
    type_model, type_input_size, type_classes
):
    # Preprocess for each model's input size
    concern_img = preprocess_image(img_path, concern_input_size)
    type_img = preprocess_image(img_path, type_input_size)

    # Predict skin concern
    concern_probs = concern_model.predict(concern_img)
    concern_index = np.argmax(concern_probs)
    concern_confidence = float(np.max(concern_probs))
    predicted_concern = concern_classes[concern_index]

    # Predict skin type
    type_probs = type_model.predict(type_img)
    type_index = np.argmax(type_probs)
    type_confidence = float(np.max(type_probs))
    predicted_type = type_classes[type_index]

    return {
        "skin_concern": {
            "class": predicted_concern,
            "confidence": concern_confidence
        },
        "skin_type": {
            "class": predicted_type,
            "confidence": type_confidence
        }
    }

In [12]:
result = predict_skin_profile_dual_input(
    img_path="images/face_image_1.jpg",
    concern_model=skin_concern_model,
    concern_input_size=(128, 128),
    concern_classes=skin_concern_classes,
    type_model=skin_type_model,
    type_input_size=(150, 150),
    type_classes=skin_type_classes
)

print(f"Skin Concern: {result['skin_concern']['class']} ({result['skin_concern']['confidence']:.2f})")
print(f"Skin Type: {result['skin_type']['class']} ({result['skin_type']['confidence']:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 701ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Skin Concern: redness (0.64)
Skin Type: dry (0.67)
